In [10]:
import pandas as pd
AppReview_df = pd.read_csv('./dataset/AWARE_Comprehensive.csv')
AppReview_df.head()

,domain,app,review_id,sentence_id,title,review,sentence,rating,is_opinion,category,term,from,to,sentiment
0,productivity,things-3,c9274c0a-a120-4e09-816b-7a8ba3a16634,00808934-e8b9-42fa-b37f-cfeac234bbdd,Difficult to update from Things 2,This new version of Things has an entirely dif...,This new version of Things has an entirely dif...,3,True,usability,new version,6.0,17.0,positive
1,productivity,notability,e633e20a-07c1-4a5e-80b1-b104b6cf6a61,00a8d4a4-9c8e-4d1c-9085-ffd1f62ae039,Great app,I have been using this app for over 3 years no...,All those contractors were blown away by how e...,5,False,NaN,NaN,NaN,NaN,NaN
2,productivity,microsoft-word,506230e3-cc98-4233-be40-89e52d53990c,011cd77b-ebbd-4589-af12-9792975b02b9,Same word. Same problem,A lot of people use word because its very stan...,"I tell them what happens, and they say “so is ...",1,True,general,NaN,NaN,NaN,negative
3,productivity,things-3,69d44a5e-218f-4f55-8a99-6cca55d43ca1,014a7d01-f6c0-408a-897b-f6b36cdd8543,Incredible Planner for Students,I was originally skeptical on paying $7 on a t...,"The ease of use, simplicity, and great functio...",5,True,effectiveness,functionality,40.0,53.0,positive
4,productivity,notability,3d0b634c-d402-47e8-ba7d-bf6209fed826,01539a8d-ebde-4cfe-9b5b-3fa10f49fa3e,Recording and syncing,This used to be my go to app for note taking. ...,I’m disappointed that even after a year they h...,1,True,cost,money,95.0,100.0,negative


In [11]:
df_x = AppReview_df['review']
df_y = AppReview_df['domain']

In [4]:
import gensim.parsing.preprocessing as gsp
filters = [
           gsp.strip_tags, 
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.strip_numeric,
           gsp.remove_stopwords, 
           gsp.strip_short, 
           gsp.stem_text
          ]
def clean_text(s):
    s = s.lower()
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return s

In [5]:
# tf-idf encode
from gensim import utils
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer

class Text2TfIdfTransformer(BaseEstimator):

    def __init__(self):
        self._model = TfidfVectorizer()
        pass

    def fit(self, df_x, df_y=None):
        df_x = df_x.apply(lambda x : clean_text(x))
        self._model.fit(df_x)
        return self

    def transform(self, df_x):
        return self._model.transform(df_x)

In [12]:
tfidf_transformer = Text2TfIdfTransformer()
tfidf_vectors = tfidf_transformer.fit(df_x).transform(df_x)# 给新闻内容编码，

In [13]:
tfidf_vectors.shape

(11321, 7827)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
pl_log_reg_tf_idf = Pipeline(steps=[('tfidf',Text2TfIdfTransformer()),
                             ('log_reg', LogisticRegression(multi_class='multinomial', solver='saga', max_iter=100))])
scores = cross_val_score(pl_log_reg_tf_idf, df_x, df_y, cv=5,scoring='accuracy')
print('Accuracy for Tf-Idf & Logistic Regression: ', scores.mean())

import xgboost as xgb

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_y1 = le.fit_transform(df_y)# 给新闻标签编码，按照字母顺序编码

pl_xgb_tf_idf = Pipeline(steps=[('tfidf',Text2TfIdfTransformer()),
                         ('xgboost', xgb.XGBClassifier(objective='multi:softmax'))])
scores = cross_val_score(pl_xgb_tf_idf, df_x, df_y1, cv=5)
print('Accuracy for Tf-Idf & XGBoost Classifier : ', scores.mean())

Accuracy for Tf-Idf & Logistic Regression:  0.9609572617571119
Accuracy for Tf-Idf & XGBoost Classifier :  0.9858671674506041


Accuracy for Tf-Idf & Logistic Regression:  0.9609572617571119
Accuracy for Tf-Idf & XGBoost Classifier :  0.9858671674506041
结果感觉会过拟合，他计算的时候并没有划分训练集测试集等。